# Steel-wire data reconstruction demo

Reproduce all steel-wire data figures.

Tested with CIL version 21.0.0 on Linux.

In [ ]:
# Import all CIL components needed
from cil.framework import ImageData, ImageGeometry
from cil.framework import AcquisitionGeometry, AcquisitionData
from cil.framework import BlockDataContainer

from cil.optimisation.algorithms import CGLS, GD, FISTA, PDHG
from cil.optimisation.operators import BlockOperator, GradientOperator, IdentityOperator, \
                                       GradientOperator, FiniteDifferenceOperator
from cil.optimisation.functions import IndicatorBox, MixedL21Norm, L2NormSquared, \
                                       BlockFunction, L1Norm, LeastSquares, \
                                       OperatorCompositionFunction, TotalVariation, \
                                       ZeroFunction

# CIL Processors
from cil.processors import CentreOfRotationCorrector, Slicer, TransmissionAbsorptionConverter

from cil.utilities.display import show2D

# Import from cil.plugins.astra
from cil.plugins.astra.processors import FBP, AstraBackProjector3D
from cil.plugins.astra.operators import ProjectionOperator, AstraProjector3DSimple

# All external imports
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import scipy

Load demo data set and display the first raw projection

In [ ]:
from cil.utilities.dataexample import SYNCHROTRON_PARALLEL_BEAM_DATA
data_sync = SYNCHROTRON_PARALLEL_BEAM_DATA.get()

scale = data_sync.get_slice(vertical=20).mean()
data_sync = data_sync/scale

data_sync = TransmissionAbsorptionConverter()(data_sync)

data_sync = CentreOfRotationCorrector.xcorrelation(slice_index='centre')(data_sync)

data90 = Slicer(roi={'angle':(0,90), 
                     'horizontal':(20,140,1)})(data_sync)

data90.reorder(order='astra')

ag90 = data90.geometry
ig = ag90.get_ImageGeometry()

recon90 = FBP(ig, ag, device='gpu')(data90)

sx = 44
sz = 103
ca1 = -0.01
ca2 =  0.11

show2D(recon90.get_slice(horizontal_x=sx), cmap='inferno', fix_range=(ca1,ca2), origin='upper-left')
show2D(recon90.get_slice(vertical=sz)   , cmap='inferno', fix_range=(ca1,ca2), origin='upper-left')

data15 = Slicer(roi={'angle': (0,90,6)})(data90)

ag = data15.geometry

recon15 = FBP(ig, ag, device='gpu')(data15)

show2D(recon15.get_slice(horizontal_x=sx), cmap='inferno', fix_range=(ca1,ca2), origin='upper-left')
show2D(recon15.get_slice(vertical=sz)   , cmap='inferno', fix_range=(ca1,ca2), origin='upper-left')

In [ ]:
A = ProjectionOperator(ig, ag)

In [ ]:
x0 = ig.allocate(0.0)
b = data15

In [ ]:
myCGLS = CGLS(x_init=x0, operator=A, data=b, max_iteration=1000)

myCGLS.run(2,verbose=True)

show2D(myCGLS.solution.get_slice(horizontal_x=sx), cmap='inferno', fix_range=(ca1,ca2), origin='upper-left')
show2D(myCGLS.solution.get_slice(vertical=sz)    , cmap='inferno', fix_range=(ca1,ca2), origin='upper-left')

In [ ]:
myCGLS.run(18,verbose=1)

show2D(myCGLS.solution.get_slice(horizontal_x=sx),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')
show2D(myCGLS.solution.get_slice(vertical=sz),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')

In [ ]:
myCGLS.run(180,verbose=True)

show2D(myCGLS.solution.get_slice(horizontal_x=sx),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')
show2D(myCGLS.solution.get_slice(vertical=sz),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')

In [ ]:
myCGLS.run(500,verbose=True)
show2D(myCGLS.solution.get_slice(horizontal_x=sx),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')
show2D(myCGLS.solution.get_slice(vertical=sz),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')

In [ ]:
plt.loglog(myCGLS.objective)

In [ ]:
f1 = 0.5*LeastSquares(A, b)

myGD_LS = GD(x_init=x0, objective_function=f1, step_size=None, max_iteration=100000, update_objective_interval = 10)
myGD_LS.run(3000, verbose=1)

In [ ]:
plt.loglog(myCGLS.iterations, myCGLS.objective)
plt.loglog(myGD_LS.objective)
len(myCGLS.objective)

In [ ]:
f1 = LeastSquares(A, b)

D = GradientOperator(ig)
alpha = 1.0
f2 = OperatorCompositionFunction(L2NormSquared(),D)
f = f1 + (alpha**2)*f2

myGD = GD(x_init=x0, objective_function=f, step_size=None, max_iteration=1000, update_objective_interval = 10)
myGD.run(1000, verbose=1)

In [ ]:
plt.figure()
show2D(myGD.solution.get_slice(horizontal_x=sx),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')
plt.figure()
show2D(myGD.solution.get_slice(vertical=sz),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')

In [ ]:
F = LeastSquares(A, b)
G = IndicatorBox(lower=0.0)
myFISTANN = FISTA(f=F, g=G, x_init=x0, max_iteration=1000)
myFISTANN.run(300, verbose=1)

In [ ]:
plt.figure()
show2D(myFISTANN.solution.get_slice(horizontal_x=sx),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')
plt.figure()
show2D(myFISTANN.solution.get_slice(vertical=sz),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')

In [ ]:
a1 = 30
G = a1*L1Norm()

myFISTAL1 = FISTA(f=F, g=G, x_init=x0, max_iteration=1000, update_objective_interval=10)

In [ ]:
myFISTAL1.run(300,verbose=1)

In [ ]:
plt.figure()
show2D(myFISTAL1.solution.get_slice(horizontal_x=sx),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')
plt.figure()
show2D(myFISTAL1.solution.get_slice(vertical=sz),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')

In [ ]:
GTV = 0.02*TotalVariation()

In [ ]:
myFISTATV = FISTA(f=F, g=GTV, x_init=x0 ,max_iteration=1000)

In [ ]:
myFISTATV.run(2,verbose=1)

In [ ]:
plt.figure()
show2D(myFISTATV.solution.get_slice(horizontal_x=sx),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')
plt.figure()
show2D(myFISTATV.solution.get_slice(vertical=sz),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')

In [ ]:
alpha = 0.02
F = BlockFunction(L2NormSquared(b=b), alpha*MixedL21Norm())
K = BlockOperator(A, GradientOperator(ig))
G = ZeroFunction()
myPDHG = PDHG(f=F, g=G, operator=K, max_iteration=10000)

In [ ]:
myPDHG.run(5000,verbose=2)

In [ ]:
plt.figure()
show2D(myPDHG.solution.get_slice(horizontal_x=sx),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')
plt.figure()
show2D(myPDHG.solution.get_slice(vertical=sz),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')

In [ ]:
plt.figure()
plt.loglog(np.arange(1,len(myFISTATV.objective)),  myFISTATV.objective[1:])
plt.loglog(np.arange(1,len(myPDHG.objective)),myPDHG.objective[1:])
plt.loglog(np.arange(1,len(myPDHG.objective)),myPDHG.dual_objective[1:])
plt.loglog(np.arange(1,len(myPDHG.objective)),np.array(myPDHG.objective[1:])-np.array(myPDHG.dual_objective[1:]))
plt.ylim((1e0,1e5))
plt.legend(['FISTA','PDHG primal','PDHG dual','PDHG gap'])
plt.grid()
plt.xlabel('Number of iterations')
plt.ylabel('Objective value')